# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [69]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

data_path = 'C:/Users/USER/Desktop/Github/ML100Days/homework/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived'] #目標變數
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1) #從訓練資料中去除ID和目標變數
df_test = df_test.drop(['PassengerId'] , axis=1) #從測試資料中去除ID
#去除ID是因為要合併
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [70]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []

for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [71]:
#更動原資料框之前，先備份:
df_backup = df

In [72]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [73]:
#先檢查空值在哪:
df.isnull().sum()
#df.isnull().sum().sort_values(ascending = False).head()

Pclass      0
Age       263
SibSp       0
Parch       0
Fare        1
dtype: int64

In [74]:
# 空值補 -1, 做羅吉斯迴歸 

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

df_m1 = df.fillna(-1)

train_X = df_m1[:train_num]

estimator = LogisticRegression(solver='liblinear')

cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6960299128976762

補-1應該會有問題，因為空值欄位是年齡和票價

In [75]:
df['Age'].describe()

count    1046.000000
mean       29.881138
std        14.413493
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: Age, dtype: float64

In [76]:
df['Fare'].describe()

count    1308.000000
mean       33.295479
std        51.758668
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

In [77]:
#空值改補平均值看看:

Average_Age = df['Age'].mean()
print(Average_Age)

#empty_index = df['Age'][df['Age'].isna()==True]
#empty_index.index

df_m2 = df['Age'].fillna(Average_Age)
print(df_m2)

Average_Fare = df['Fare'].mean()
print(Average_Fare)

df_m3 = df['Fare'].fillna(Average_Fare)
print(df_m3)


29.881137667304014
0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
413    29.881138
414    39.000000
415    38.500000
416    29.881138
417    29.881138
Name: Age, Length: 1309, dtype: float64
33.2954792813456
0        7.2500
1       71.2833
2        7.9250
3       53.1000
4        8.0500
         ...   
413      8.0500
414    108.9000
415      7.2500
416      8.0500
417     22.3583
Name: Fare, Length: 1309, dtype: float64


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 5 columns):
Pclass    1309 non-null int64
Age       1046 non-null float64
SibSp     1309 non-null int64
Parch     1309 non-null int64
Fare      1308 non-null float64
dtypes: float64(2), int64(3)
memory usage: 61.4 KB


In [79]:
df_backup2 = df

In [80]:
df['Age'] = df_m2
df['Fare'] = df_m3

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 5 columns):
Pclass    1309 non-null int64
Age       1309 non-null float64
SibSp     1309 non-null int64
Parch     1309 non-null int64
Fare      1309 non-null float64
dtypes: float64(2), int64(3)
memory usage: 61.4 KB


In [82]:
train_X = df[:train_num]

estimator = LogisticRegression(solver='liblinear')

cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6981761033723469

從0.6960到0.6981，小幅增進

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [86]:
#使用原值:

train_X = df[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6981761033723469

In [88]:
#使用最小最大化:

df_temp = MinMaxScaler().fit_transform(df)

train_X = df_temp[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6993501991462476

In [89]:
#使用標準化:

df_temp2 = StandardScaler().fit_transform(df)

train_X = df_temp2[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6959413955734954

用最小最大化搭配羅吉斯模型，在這裡可以達到三者中最好的成績0.6993